In [90]:
# Setup
import numpy as np
import pandas as pd
import random

## Helper functions

In [110]:
insurance_value = 0

# Basic strategy approach
def basic_strategy(player_hand, visible_dealer_card):
    """Returns a player action.
    
    The function takes two arguments, and based on their value turns different events:
    "BlackJack", "Split", "Double", "Hit", "Stand"

  
    player_hand = a list of card values representing the player's hand
    visible_dealer_card = the card face up in the dealer's hand
"""
    assert len(player_hand) >= 2, \
    "The player should have at least two cards in their hand."
    print(player_hand)
    cards_count = len(player_hand)
    count_of_aces = player_hand.count(1)
    hard_hand = sum(player_hand)
    soft_hand = hard_hand if hard_hand > 11 else hard_hand + (10 * count_of_aces)
    dealer_advantage_cards = [1, 7, 8, 9, 10]
    if hard_hand <= 11:
        soft_hand= hard_hand + (10 * count_of_aces)
    if cards_count == 2 and soft_hand == 21:
        return 'Blackjack!'
    if soft_hand >= 19:
        return 'The player stands.'
    if player_hand == [9, 9] and visible_dealer_card not in [1, 7, 10]:
        return 'The player splits.'
    if hard_hand >= 17:
        return 'The player stands.'

    if cards_count == 2:
        if player_hand[0] == player_hand[1]:
            split_card = player_hand[0]
            if split_card == 1:
                return 'The player splits.'
            if split_card == 8 and visible_dealer_card != 1:
                return 'The player splits.'
            if split_card in [2, 3, 7] and visible_dealer_card not in [1, 8, 9, 10]:
                return 'The player splits.'
            if split_card == 6 and visible_dealer_card not in dealer_advantage_cards:
                return 'The player splits.'
            if split_card == 4 and visible_dealer_card in [5,6]:
                return 'The player splits.'

        if hard_hand == soft_hand:
            if hard_hand == 11 and visible_dealer_card != 1:
                return 'The player doubles.'
            if hard_hand == 10 and visible_dealer_card not in [1, 10]:
                return 'The player doubles.'
            if hard_hand == 9 and visible_dealer_card not in ([2] + dealer_advantage_cards):
                return 'The player doubles.'
            if hard_hand <= 11:
                return 'The player hits.'
            if hard_hand == 12 and visible_dealer_card <= 3:
                return 'The player hits.'
            if visible_dealer_card in dealer_advantage_cards:
                return 'The player hits.'
            else:
                return 'The player stands.'

        if hard_hand != soft_hand:
            if soft_hand == 18:
                if visible_dealer_card not in ([2] + dealer_advantage_cards):
                    return 'The player doubles.'
                if visible_dealer_card in [2, 7, 8]:
                    return 'The player stands.'
            if soft_hand == 17 and visible_dealer_card in [3, 4, 5, 6]:
                return 'The player doubles.'
            if soft_hand in [15, 16] and visible_dealer_card in [4, 5, 6]:
                return 'The player doubles.'
            if soft_hand in [13, 14] and visible_dealer_card in [5, 6]:
                return 'The player doubles.'
            else:
                return 'The player hits.'

    if cards_count > 2:
        if hard_hand == soft_hand:
            if hard_hand <= 11:
                return 'The player hits.'
            if hard_hand == 12 and visible_dealer_card <= 3:
                return 'The player hits.'
            if visible_dealer_card in dealer_advantage_cards:
                return 'The player hits.'
            else:
                return 'The player stands.'

        if soft_hand <= 17:
            return 'The player hits.'
        if soft_hand == 18 and visible_dealer_card in [1, 9, 10]:
            return 'The player hits.'
        else:
            return 'The player stands.'
        

def blackjack_hand_result(bet=20, player_hand=None, dealer_card=None, event=None, bonus=1, play_with_insurance=False):
    # Manage split aces event
    if player_hand == [1]:
        split_ace_flag == True
    else:
        split_ace_flag = False
        
    dealer_cards = dealer_card if type(dealer_card) == list else [dealer_card]
    
    if play_with_insurance:
        if card_count == 2 and visible_dealer_card == 1:
            insurance += bet / 2
    insurance_money = 0        
    
    # If player can split on aces
    if split_ace_flag == True:
        dealer_hit_count = 0
        while True:
            # The dealer hits
            new_card = random.choice(deck)
            dealer_cards.append(new_card)
            # If we play with insurance, check if player gets insurance money
            if play_with_insurance and insurance != 0 and dealer_hit_count == 0 and visible_dealer_card == 1 and new_card == 10:
                insurance_money = insurance * 2
                insurance = 0
            dealer_hit_count += 1
            dealer_hand = sum(dealer_cards)
            # Manage soft hand
            dealer_soft_hand = dealer_hand
            if dealer_hand <= 11 and 1 in dealer_cards:
                dealer_soft_hand += 10
            # House wins no matter what if it hits 21
            if dealer_soft_hand == 21 and len(dealer_cards) == 2:
                return -bet + insurance_money
            
            # If it's advantageous for player, use soft score instead
            score = sum(player_hand)
            if score <= 11 and 1 in player_hand:
                score += 10
                
            # Dealer has to stand when they hit 17
            if dealer_soft_hand >= 17:
                # Player wins if house busts
                if dealer_soft_hand > 21:
                    return bet
                # Player wins if house scores lower
                if score > dealer_soft_hand:
                    return bet + insurance_money
                # Nobody wins if there's a tie
                if score == dealer_soft_hand:
                    return 0 + insurance_money
                # Player loses if house scores higher
                if score < dealer_soft_hand:
                    return -bet + insurance * 2

    # If player can't split on aces
    if split_ace_flag == False:
        # Assessing player blackjack
        if basic_strategy(player_hand, dealer_card) == 'Blackjack!':
            # Assessing house blackjack
            dealer_cards.append(random.choice(deck))
            if 1 in (dealer_cards):
                # House has blackjack, tie
                if sum(dealer_cards) == 11:
                    if dealer_card == 1:
                        insurance_money = insurance * 2
                        insurance = 0
                        return 0 + insurance_money
                    else:
                        return 0
            # Player wins a bonus
            else:
                return bet * bonus
        
        # Assess what to do after first hit
        if event != "The player hits for the first time.":
            # Player hits again
            if event == 'The player hits.':
                player_hand.append(random.choice(deck))
                return blackjack_hand_result(bet, player_hand, dealer_card, event="The player hits for the first time.")
            
            # Player doubles
            if event == 'The player doubles.':
                if len(player_hand) == 2:
                    player_hand.append(random.choice(deck))
                    bet = bet*2
                    if sum(player_hand) > 21:
                        if dealer_card == 1 and random.choice(deck) == 11 and insurance != 0:
                            insurance_money = insurance * 2
                            insurance = 0
                            return -bet + insurance_money
                        else:
                            return -bet
                else:
                    return 'The player hits.'
                
            # Player splits
            if event == 'The player splits.':
                if len(player_hand) == 2: 
                    if player_hand[0] == player_hand[1]:
                        split_hand_left = blackjack_hand_result(bet=bet, player_hand=[player_hand[0]], dealer_card=dealer_card, event='The player splits.')
                        split_hand_right += blackjack_hand_result(bet=bet, player_hand=[player_hand[0]], dealer_card=dealer_card, event='The player splits.')
                        outcome = split_hand_left + split_hand_right
                        return outcome 
                    else:
                        return blackjack_hand_result(bet=bet, player_hand=player_hand, dealer_card=dealer_card)
                else:
                    return blackjack_hand_result(bet=bet, player_hand=player_hand, dealer_card=dealer_card)

        else:
            # Player hits
            while basic_strategy(player_hand, dealer_card) == 'The player hits.':
                player_hand.append(random.choice(deck))
                # Player loses if they bust
                if sum(player_hand) > 21:
                    if dealer_card == 1 and random.choice(deck) == 11 and insurance != 0:
                        insurance_money = insurance * 2
                        insurance = 0
                        return -bet + insurance_money
                    else:
                        return -bet

            # Player doubles
            if basic_strategy(player_hand, dealer_card) == 'The player doubles.':
                player_hand.append(random.choice(deck))
                bet = bet * 2
                # Player loses if they bust
                if dealer_card == 1 and random.choice(deck) == 11 and insurance != 0:
                    insurance_money = insurance * 2
                    insurance = 0
                    return -bet + insurance_money
                else:
                    return -bet
            # Player splits
            if basic_strategy(player_hand, dealer_card) == 'The player splits.':
                split_hand_left = blackjack_hand_result(bet, [player_hand[0]], dealer_card) 
                split_hand_right = blackjack_hand_result(bet, [player_hand[0]], dealer_card)
                outcome = split_hand_left + split_hand_right
                return outcome
        
        dealer_hit_count = 0
        while True:
            # The dealer hits
            new_card = random.choice(deck)
            dealer_cards.append(new_card)
            # If we play with insurance, check if player gets insurance money
            if play_with_insurance and insurance != 0 and dealer_hit_count == 0 and visible_dealer_card == 1 and new_card == 10:
                insurance_money = insurance * 2
                insurance = 0
            dealer_hit_count += 1
            dealer_hand = sum(dealer_cards)
            # Manage soft hand
            dealer_soft_hand = dealer_hand
            if dealer_hand <= 11 and 1 in dealer_cards:
                dealer_soft_hand += 10
                
            # House wins no matter what if it hits 21
            if len(dealer_cards) == 2 and dealer_soft_hand == 21:
                return -bet + insurance_money
            
            # If it's advantageous for player, use soft score instead
            score = sum(player_hand)
            if score <= 11 and 1 in player_hand:
                score += 10

            # Dealer has to stand when they hit 17
            if dealer_soft_hand >= 17:
                # Player wins if house busts
                if dealer_soft_hand > 21:
                    return bet + insurance_money
                # Player wins if house scores lower
                if score > dealer_soft_hand:
                    return bet + insurance_money
                # Nobody wins if there's a tie
                if score == dealer_soft_hand:
                    return 0 + insurance_money
                # Player loses if house scores higher
                if score < dealer_soft_hand:
                    return -bet + insurance_money
                
def simulation(number_of_simulations=1000000, player_hand=None, visible_dealer_card=None, bet=10, event="The player hits for the first time."):
    profit = 0
    cards_count = len(player_hand)
    for i in range(number_of_simulations):
        profit += blackjack_hand_result(bet=bet, player_hand=player_hand[:cards_count], dealer_card=visible_dealer_card, event=event)
        print(profit)
    return profit

In [116]:
# Initialize a deck of cards
deck = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10])

# Initialize a first hand
first_hand = [random.choice(deck), random.choice(deck)]

# Initialize a visible dealer card
visible_dealer_card = random.choice(deck)

simulation(number_of_simulations=10, player_hand=first_hand, visible_dealer_card=visible_dealer_card, bet=10, event="The player hits for the first time.")

[8, 3]
[8, 3]
[8, 3]
-20
[8, 3]
[8, 3]
[8, 3]
-40
[8, 3]
[8, 3]
[8, 3]
-60
[8, 3]
[8, 3]
[8, 3]
-80
[8, 3]
[8, 3]
[8, 3]
-100
[8, 3]
[8, 3]
[8, 3]
-120
[8, 3]
[8, 3]
[8, 3]
-140
[8, 3]
[8, 3]
[8, 3]
-160
[8, 3]
[8, 3]
[8, 3]
-180
[8, 3]
[8, 3]
[8, 3]
-200


-200